# Librairies

In [1]:
import os
import json
import datetime 
import random

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.utils import Sequence, to_categorical

print(tf.test.is_gpu_available())

min_frame = 5

True


# Data

In [2]:
list_IDs = {}
with open("data/dataset.json", "r") as file:
    list_IDs = json.load(file)

In [3]:
class NTUSequence(Sequence):
    
    def __init__(self, list_IDs, path, batch_size=32, one_hot=True, min_frame=min_frame):
        # Removed unused vars (self.x, max_frame)
        self.x = list_IDs
        self.batch_size = batch_size
        self.path = path
        self.min_frame = min_frame
        if one_hot:
            self.one_hot_encode()

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        # idx: index of the data
    
        # Should be the same as normal
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        
        #list of batch_x files 
        F = [np.load(self.path + ID + '.npy')[:, :, :, np.newaxis] for ID in batch_x]
        
        if not(self.min_frame is None):
            X, Y = [], []
            
            for x in F:
                
                if min_frame >= x.shape[0] - 1:
                    X.append(min_frame * x[0])
                    Y.append(x[0])
                    
                else:
                    f0 = random.randint(min_frame, x.shape[0] - 1)

                    X.append(x[f0 - min_frame : f0])
                    Y.append(x[f0])

                
            X = np.stack(X)
            Y = np.stack(Y)
#         print("X=")
#         print(X.shape)
#         print("Y=")
#         print(Y.shape)
        
        return X, Y

    def one_hot_encode(self):
#         labels = [val for val in self.y.values()]
#         ids = [key for key in self.y.keys()]
#         labels = to_categorical(labels)
#         self.y = {ID: label for ID, label in zip(ids, labels)}
        pass

In [4]:
trainset = NTUSequence(list_IDs["train"], path="D:/PFE/train/")
testset = NTUSequence(list_IDs["validation"], path="D:/PFE/test/")

# Architecture

In [5]:
convnet = models.Sequential([
    
    layers.Conv2D(64, 3, activation="relu", padding="same", input_shape=(25, 25, 1)),
    layers.Conv2D(64, 3, activation="relu", padding="same"),
    layers.Conv2D(64, 3, activation="relu", padding="same"),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.MaxPooling2D((2,2)),
                        
    layers.Conv2D(512, 3, activation="relu", padding="same"),
    layers.Conv2D(512, 3, activation="relu", padding="same"),
    layers.MaxPooling2D((2,2)),
    
    layers.Flatten()
                        ])

In [6]:
convnet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 25, 25, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 25, 64)        36928     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 12, 12, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 128)         0

In [7]:
predictor = models.Sequential([
    layers.Input((None, 25, 25, 1)),
    layers.TimeDistributed(convnet),
    
    layers.LSTM(9, activation="relu"),
    layers.Reshape((3,3,1)),
    
    layers.Conv2DTranspose(1, 3, strides=2, activation="relu", padding="same"),
    layers.Conv2DTranspose(1, 3, strides=2, activation="relu", padding="same"),
    layers.Conv2DTranspose(1, 3, strides=2, activation="sigmoid"),
    
    layers.Reshape((25,25))
])

In [8]:
predictor.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, None, 512)         4721152   
_________________________________________________________________
lstm (LSTM)                  (None, 9)                 18792     
_________________________________________________________________
reshape (Reshape)            (None, 3, 3, 1)           0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 6, 6, 1)           10        
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 12, 12, 1)         10        
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 25, 25, 1)         10        
_________________________________________________________________
reshape_1 (Reshape)          (None, 25, 25)           

In [9]:
predictor.compile(loss="mean_squared_logarithmic_error", optimizer="adam", metrics=["mse"])

# Entraînement

In [ ]:
log_dir="logs\\predictor\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
print(log_dir)

history = predictor.fit_generator(trainset, 
                     epochs=1,
                   callbacks=[tensorboard_callback])

logs\predictor\fit\20200128-195450
 623/1157 [===============>..............] - ETA: 14:58 - loss: 0.0324 - mse: 0.0617